<a href="https://colab.research.google.com/github/zeroxy/lotto/blob/master/lotto_by_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

import tensorflow as tf
from datetime import datetime as dt
from joblib import Parallel, delayed, cpu_count
from itertools import permutations as comb
import numpy as np
np.set_printoptions(precision=8, suppress=True, linewidth=120, threshold=np.inf)
np.seterr(all="ignore")

with np.load('gdrive/My Drive/lotto/lottos_db.npz') as a :
#with np.load('lottos_db.npz') as a :
    lottos = a['lottos']
    bonus = a['add_lottos']

data = lottos - bonus
neg_data = 1-data
cnted_lot = np.ones_like(neg_data)
cnted_lot[0] *= neg_data[0]
for  i in range(1, neg_data.shape[0]):
    cnted_lot[i] += cnted_lot[i-1]
    cnted_lot[i] *= neg_data[i]
print(f'{cnted_lot[:2,:30]}\n{data[:2,:30]}')
print(f'{cnted_lot[-1:,:30]}\n{data[-1:,:30]}')

traindataset = 810
#testidx = np.random.choice(data.shape[0]-1, data.shape[0]-1-traindataset, replace=False)
#trainidx = np.delete(np.arange(data.shape[0]-1),testidx)
trainidx = np.arange(traindataset)
testidx = np.arange(traindataset, data.shape[0]-1)

np.random.shuffle(trainidx)
np.random.shuffle(testidx)

#trainidx=np.reshape(trainidx, (-1, traindataset))

def correct_test(pb, bias, y_, testtime):
    count = np.zeros((pb.shape[0],7),dtype=np.int32)
    pb = pb / np.sum(pb, axis=1).reshape((-1,1))
    pb += bias
    pb = pb / np.sum(pb, axis=1).reshape((-1,1))
    for i in range(pb.shape[0]):
        real = np.where(y_[i])[0]
        for f in range(testtime):
            virtual = np.random.choice(45,6, replace=False, p = pb[i])
            include = np.sum(np.in1d(virtual, real))
            count[i, include]+=1
    return count

cpucnt = cpu_count()
multipool = Parallel(n_jobs=(cpucnt*2), backend='multiprocessing')
poolfn = delayed(correct_test)

money = np.asarray([0,0,0,5,50,1700,50000])
testtimes = 1000

bias_sample = np.arange(0.000, 0.00002, 0.0001)
lr = 0.0002

data_desc = f'train = {trainidx.shape}, test = {testidx.shape}, all data = {lottos.shape}'
model_desc = f'lr = {lr}, bias sample = {bias_sample}'
print(model_desc+'\n'+data_desc)

x = tf.placeholder(tf.float32, shape=(None, 45))
y = tf.placeholder(tf.float32, shape=(None, 45))

channels = [128,256,1024]
layer_type = []
for i in channels:
    layer_type.append(('fc',i,0.01))
    layer_type.append(('fc',i,0.))
layer_type.append(('dr',0.5,0.5))
layer_type.append(('dr',0.7,0.7))
print(layer_type)

nets = {}
combnum = 4
def net(layer, is_training, reuse):
    #layer = tf.expand_dims(layer, -1, name="expand")
    #flatten_layer = tf.layers.flatten(layer, name="flatten")
    for e in range(combnum):
        li = list(comb(layer_type,e))
        print(f'{e} ==> {len(li)}')
        for row in li:
            key = 'lot'+"_".join([f'{r[0]}{r[1]}-{r[2]}' for r in row])
            with tf.variable_scope(key, reuse=reuse):
                nets[key]= tf.layers.flatten(layer, name="flatten")
                for idx,(ltype,chan,rate) in enumerate(row):
                    if ltype=='fc':
                        if rate == 0.:
                            nets[key] = tf.layers.dense(nets[key], chan , activation=tf.nn.sigmoid, trainable=is_training, name=f"fc{idx}")
                        else:
                            nets[key] = tf.layers.dense(nets[key], chan , activation=tf.nn.sigmoid, trainable=is_training, name=f"fc{idx}",
                                                      kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=rate))
                    elif ltype=='dr':
                        nets[key] = tf.layers.dropout(nets[key], rate=rate , training=is_training, name=f"dr{idx}")
                nets[key] = tf.layers.dense(nets[key], 45 , activation=tf.nn.sigmoid, trainable=is_training, name=f"fc_fin")
    return nets

trainnets = net(x, True, False)
testnets = net(x,False, True)
loss = {}
trainjob = {}
starttime = dt.now()

for idx,k in enumerate(nets.keys()):
    loss[k] = tf.losses.mean_squared_error(y, trainnets[k]) + tf.losses.get_regularization_loss(scope=k, name=f'{k}_reg_loss')
    trainjob[k] = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(loss[k])
    if idx%10 == 0:
        print(f'\r{idx} / {len(nets.keys())}   {dt.now() - starttime}', end='')
    #trainjob[k] = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss[k])



config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
print("######")
#for pb_bias in bias_sample: #[0.0015, 0.002,0.0025, 0.003,0.0035, 0.004, 0.005, 0.006]: #[0.02, 0.01, 0.005, 0.002, 0.001]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    print(f'\n\n{starttime}  -  {dt.now() - starttime}')
    for jj in range(30):
        for ii in range(1000):
            trainloss ,_ = sess.run([loss, trainjob], feed_dict={x:cnted_lot[trainidx], y:data[trainidx+1]})
            #if (k+1) % 500 == 0 :
            print(f'\r== {ii}==  {dt.now()-starttime}',end='')
        #for k in nets.keys():
        #    print(f'== {k} == loss: {trainloss[k]}')
        print()

        #### test phase!!!!! #####
        testresult,x_, y_ = sess.run([testnets,x, y], feed_dict={x:cnted_lot[testidx], y:data[testidx+1]})
        realtest,realx_, realy_ = sess.run([testnets,x,  y], feed_dict={x:cnted_lot[-1:], y:data[-1:]})
        print(f'== {jj+1} ==>>> {(realx_.astype(np.int32))[0,:5]}==  {dt.now()-starttime}  {dt.now()}')
        count = np.zeros((len(nets.keys()),len(testidx),7),dtype=np.int32) 
        for pb_bias in bias_sample:
            returns = multipool(poolfn(testresult[k],pb_bias,y_, testtimes) for kidx,k in enumerate(nets.keys()))
            for r in range(len(returns)):
                count[r] = returns[r]
            meancnt = np.mean(count, axis=1)
            report = ""
            bestreport = ""
            for kidx, k in enumerate(nets.keys()):
                tempreport = f'{k} == step : {jj+1}, pb bias :{pb_bias} ==>> {np.around(np.sum(meancnt[kidx, 3:]),2)}  '
                tempreport += f'{np.around(np.sum(meancnt[kidx]*money),2)} won / {testtimes}  *****  {dt.now()-starttime}\n{np.around(meancnt[kidx],2)}\n'

                #### real test phase!!!!! #####

                if np.sum(meancnt[kidx,3:])>0.0280*testtimes or np.sum(meancnt[kidx]*money)>testtimes*0.3:
                    virtual = np.zeros((10,6),dtype=np.int32)
                    for f in range(virtual.shape[0]):
                        pb = realtest[k][0]
                        pb /= np.sum(pb)
                        pb += pb_bias
                        pb /= np.sum(pb)
                        virtual[f] = np.sort(np.random.choice(45,6, replace=False, p = pb))+1
                    tempreport += f'{np.unique(virtual, axis=0)}\n'
                    tempreport += f'{realtest[k][0] }\n\n'
                    bestreport += tempreport
                else :
                    report += tempreport
            #print(f'{report}\n\n{bestreport} \n#############    {dt.now()-starttime}  {dt.now()}    #################\n')
            print(f'{bestreport} \n#############    {dt.now()-starttime}  {dt.now()}    #################\n')


Mounted at /content/gdrive
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 0. 1. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 0. 2. 1. 2. 0. 2. 2. 2. 1. 2.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
lr = 0.0002, bias sample = [0.]
train = (810,), test = (47,), all data = (858, 45)
[('fc', 128, 0.01), ('fc', 128, 0.0), ('fc', 256, 0.01), ('fc', 256, 0.0), ('fc', 1024, 0.01), ('fc', 1024, 0.0), ('dr', 0.5, 0.5), ('dr', 0.7, 0.7)]
0 ==> 1
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
1 ==> 8

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/a